In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.lite import TFLiteConverter

In [14]:
# Load landmark data
landmarks_df = pd.read_csv('../../data/HaGRID/modified/csv/train_label_balanced.csv')
landmarks_df.head()

,image_path,label,handedness,x_0,y_0,z_0,x_1,y_1,z_1,x_2,...,z_17,x_18,y_18,z_18,x_19,y_19,z_19,x_20,y_20,z_20
0,MODIFIED/call/c59f650f-5f8a-47e8-b948-f58c7b7a...,call,Right,0.754290,0.386637,-4.098337e-07,0.749103,0.344792,0.001934,0.726038,...,-0.035032,0.595950,0.367655,-0.041725,0.564724,0.370839,-0.039518,0.541878,0.370556,-0.037125
1,MODIFIED/call/0e6e38df-275c-4b54-bae2-cbadfe7d...,call,Left,0.402540,0.481069,-1.031420e-07,0.410768,0.462422,-0.001044,0.423757,...,-0.011130,0.453464,0.498888,-0.013328,0.463487,0.505605,-0.012871,0.472385,0.510082,-0.012735
2,MODIFIED/call/b2432f41-2b8e-409c-91ff-7b777fb1...,call,Right,0.674610,0.532156,-1.900930e-07,0.664562,0.495894,-0.000400,0.645847,...,-0.021909,0.598007,0.527357,-0.026023,0.591233,0.529411,-0.024737,0.587414,0.526515,-0.023360
3,MODIFIED/call/7192505e-d304-4f86-9f4b-8a537767...,call,Right,0.656416,0.334383,-4.484448e-07,0.605983,0.291761,0.001999,0.536142,...,-0.083188,0.489501,0.410172,-0.093585,0.450586,0.429055,-0.093920,0.418757,0.441578,-0.096475
4,MODIFIED/call/de9a7ee3-8baa-4289-8970-a7b796a7...,call,Left,0.340446,0.560608,6.133277e-08,0.332125,0.527111,-0.012109,0.345604,...,-0.021573,0.479623,0.556234,-0.027739,0.507211,0.556837,-0.026317,0.530148,0.557762,-0.024677


In [15]:
# Prepare features and labels
X = landmarks_df.drop(columns=['image_path', 'handedness', 'label']).values
y = landmarks_df['label'].values

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [16]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Reshape data for input
X_train = X_train.reshape((X_train.shape[0], 21, 3))
X_test = X_test.reshape((X_test.shape[0], 21, 3))

In [22]:
# Define a model
model = Sequential()
model.add(Input(shape=(21,3)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(le.classes_), activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [23]:
# Train the model
model.fit(X_train, y_train, epochs=25, validation_data=(X_test, y_test))
model_base_dir = '../../models/HaGRID/FNN2/'

# Save the model and label encoder
model.save(model_base_dir + 'model.keras')
np.save(model_base_dir + 'label_encoder_classes.npy', le.classes_)

Epoch 1/25
6696/6696 ━━━━━━━━━━━━━━━━━━━━ 3s 426us/step - accuracy: 0.5946 - loss: 1.2131 - val_accuracy: 0.8268 - val_loss: 0.4786
Epoch 2/25
6696/6696 ━━━━━━━━━━━━━━━━━━━━ 3s 423us/step - accuracy: 0.9046 - loss: 0.3472 - val_accuracy: 0.9507 - val_loss: 0.2113
Epoch 3/25
6696/6696 ━━━━━━━━━━━━━━━━━━━━ 3s 425us/step - accuracy: 0.9492 - loss: 0.2092 - val_accuracy: 0.9562 - val_loss: 0.1754
Epoch 4/25
6696/6696 ━━━━━━━━━━━━━━━━━━━━ 3s 431us/step - accuracy: 0.9551 - loss: 0.1769 - val_accuracy: 0.9640 - val_loss: 0.1440
Epoch 5/25
6696/6696 ━━━━━━━━━━━━━━━━━━━━ 3s 417us/step - accuracy: 0.9580 - loss: 0.1585 - val_accuracy: 0.9618 - val_loss: 0.1456
Epoch 6/25
6696/6696 ━━━━━━━━━━━━━━━━━━━━ 3s 420us/step - accuracy: 0.9597 - loss: 0.1478 - val_accuracy: 0.9637 - val_loss: 0.1399
Epoch 7/25
6696/6696 ━━━━━━━━━━━━━━━━━━━━ 3s 427us/step - accuracy: 0.9609 - loss: 0.1408 - val_accuracy: 0.9544 - val_loss: 0.1567
Epoch 8/25
6696/6696 ━━━━━━━━━━━━━━━━━━━━ 3s 427us/step - accuracy: 0.9614 -

In [24]:
# Save as TFLite
converter = TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open(model_base_dir + 'model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpbfq9ewl4/assets


INFO:tensorflow:Assets written to: /tmp/tmpbfq9ewl4/assets


Saved artifact at '/tmp/tmpbfq9ewl4'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 21, 3), dtype=tf.float32, name='keras_tensor_11')
Output Type:
  TensorSpec(shape=(None, 12), dtype=tf.float32, name=None)
Captures:
  126366057652944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126366057651408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126366057653328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126366057655248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126366057653136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  126366057654672: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1722472844.040099   10235 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1722472844.040110   10235 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-07-31 20:40:44.040211: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpbfq9ewl4
2024-07-31 20:40:44.040444: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-07-31 20:40:44.040451: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpbfq9ewl4
2024-07-31 20:40:44.042600: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-07-31 20:40:44.054832: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpbfq9ewl4
2024-07-31 20:40:44.059071: I tensorflow/cc/saved_model/loader.cc:462] SavedModel load for tags { serve }; Status: success: OK. Took 18863 microseconds.
